# Setup

### Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Install Dependencies

In [2]:
# Needed for converting tif to png
!sudo apt-get install imagemagick

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 liblqr-1-0 libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra
  libmagickwand-6.q16-3 libnetpbm10 libwmf0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng
  enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance
  sane-utils texlive-base-bin transfig ufraw-batch inkscape libjxr-tools
  libwmf0.2-7-gtk poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-goth

### Imports & Global Variables

In [3]:
import os
import glob


remote_code_path = "/content/drive/MyDrive/Vessels"
remote_dataset_path = "/content/drive/MyDrive/VesselNet"
local_code_path = "/content/"+os.path.basename(remote_code_path)
first_image_index = 1
last_image_index = 20

### Copy code into working directory

In [4]:
os.system(f"cp -r {remote_code_path} ./")

0

# Prepare Data

### Fetch and Parse VesselNet Data

In [15]:
# Clear the code's data directory
os.system(f"rm {local_code_path}/DRIVE/training/images/*")
os.system(f"rm {local_code_path}/DRIVE/training/1st_manual/*")

for i in range(first_image_index, last_image_index+1):
  # Create n masks (n is the number of images to train on)
  os.system(f"cp {local_code_path}/DRIVE/training/mask/20_training_mask.png {local_code_path}/DRIVE/training/mask/{i:02d}_training_mask.png")

  # Copy images and groundtruth into the code's data directory
  # And convert them from TIF to PNG
  os.system(f"convert -verbose -coalesce {remote_dataset_path}/dataset/validate/origin/{i+20:02d}_test.tif {local_code_path}/DRIVE/training/images/{i:02d}_training.png")
  os.system(f"convert -verbose -coalesce {remote_dataset_path}/dataset/validate/groundtruth/{i+20:02d}_manual1.tif {local_code_path}/DRIVE/training/1st_manual/{i:02d}_manual1.png")

  # Resize both images and groundtruth to UNet's size (512x512)
  os.system(f"ffmpeg -y -i {local_code_path}/DRIVE/training/images/{i:02d}_training.png -s 512x512 {local_code_path}/DRIVE/training/images/{i:02d}_training.png")
  os.system(f"ffmpeg -y -i {local_code_path}/DRIVE/training/1st_manual/{i:02d}_manual1.png -s 512x512 {local_code_path}/DRIVE/training/1st_manual/{i:02d}_manual1.png")


### Prepare Dataset

In [16]:
# This extracts all testing and training patches and saves them into an h5 file to be used by the network
!cd Vessels && python3 prepare_datasets_DRIVE.py

imgs max: 0.0
imgs min: 0.0
ground truth and border masks are correctly withih pixel value range 0-255 (black-white)
saving train datasets
original image: 
ground truth name: 31_manual1.png
border masks name: 31_test_mask.png
original image: 
ground truth name: 32_manual1.png
border masks name: 32_test_mask.png
original image: 
ground truth name: 34_manual1.png
border masks name: 34_test_mask.png
original image: 
ground truth name: 23_manual1.png
border masks name: 23_test_mask.png
original image: 
ground truth name: 26_manual1.png
border masks name: 26_test_mask.png
original image: 
ground truth name: 22_manual1.png
border masks name: 22_test_mask.png
original image: 
ground truth name: 29_manual1.png
border masks name: 29_test_mask.png
original image: 
ground truth name: 25_manual1.png
border masks name: 25_test_mask.png
original image: 
ground truth name: 36_manual1.png
border masks name: 36_test_mask.png
original image: 
ground truth name: 21_manual1.png
border masks name: 21_test_

# Run Training

In [17]:
!cd Vessels && python3 src/retinaNN_training.py

./lib/pre_processing.py:63: RuntimeWarning: invalid value encountered in true_divide
  imgs_normalized = (imgs-imgs_mean)/imgs_std
Traceback (most recent call last):
  File "src/retinaNN_training.py", line 156, in <module>
    inside_FOV = config.getboolean('training settings', 'inside_FOV') #select the patches only inside the FOV  (default == True)
  File "./lib/extract_patches.py", line 35, in get_data_training
    assert(np.min(train_masks)==0 and np.max(train_masks)==1)
AssertionError
